In [37]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [40]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)
    skip = getattr(module, "skip", False)

    if skip:
        print("  Marked for skip")
        continue

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 2.983549628568871,
           'expected_sample_data_size': 16.492132820191532,
           'expected_time_to_recover': 99.51339999999999,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.16078236699781137,
             'bit_redundancy': 417.56699999999995,
             'ideal_bit_efficency': 0.18986909438272778,
             'ideal_bit_redundancy': 341.3429271366321,
             'ideal_packet_efficiency': 0.9493454719136388,
             'ideal_packet_redundancy': 68.26858542732643,
             'ideal_time_to_recover': 84.26858542732643,
             'ideal_transmitted_bits': 421.3429271366321,
             'packet_efficiency': 0.8039118349890568,
             'packet_redundancy': 83.51339999999999,
             'permeability': 0.846806414285176,
             'saturation': 16.0,
             'transmitted_bits': 497.56699999999995}}
Found implementation: anfext2
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 2.9901313171508157,
           'expected_sample_data_size': 16.480302427377637,
           'expected_time_to_recover': 39.62,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.4038364462392731,
             'bit_redundancy': 118.1,
             'ideal_bit_efficency': 0.4771071893113761,
             'ideal_bit_redundancy': 87.6772050227676,
             'ideal_packet_efficiency': 2.385535946556881,
             'ideal_packet_redundancy': 17.535441004553512,
             'ideal_time_to_recover': 33.53544100455351,
             'ideal_transmitted_bits': 167.6772050227676,
             'packet_efficiency': 2.0191822311963654,
             'packet_redundancy': 23.619999999999997,
             'permeability': 0.8464270823966057,
             'saturation': 16.0,
             'transmitted_bits': 198.1}}
Found implementation: anfext2l
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 3.0063748406289843,
           'expected_sample_data_size': 16.506200981339102,
           'expected_time_to_recover': 41.201800000000006,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.3883325485779747,
             'bit_redundancy': 126.00900000000001,
             'ideal_bit_efficency': 0.4590619190225748,
             'ideal_bit_redundancy': 94.26843021598123,
             'ideal_packet_efficiency': 2.2953095951128737,
             'ideal_packet_redundancy': 18.853686043196248,
             'ideal_time_to_recover': 34.85368604319625,
             'ideal_transmitted_bits': 174.26843021598123,
             'packet_efficiency': 1.9416627428898734,
             'packet_redundancy': 25.201800000000006,
             'permeability': 0.8459262955306865,
             'saturation': 16.0,
             'transmitted_bits': 206.00900000000001}}
Found implementation: anfext2lr
  First time seen. Benchmarking.

benchmark: 100%|██████████| 10000/10000 [00:13<00:00, 754.40it/s]


...

{'basic': {'expected_sample_burst_size': 2.9921299723380215,
           'expected_sample_data_size': 16.502551914910196,
           'expected_time_to_recover': 40.6914,
           'packet_size': 5,
           'payload_bits': 78.25340432877415},
 'derived': {'bit_efficency': 0.384618884229956,
             'bit_redundancy': 125.20359567122584,
             'ideal_bit_efficency': 0.45435535270862043,
             'ideal_bit_redundancy': 93.97611572038947,
             'ideal_packet_efficiency': 2.271776763543102,
             'ideal_packet_redundancy': 18.795223144077895,
             'ideal_time_to_recover': 34.445904009832724,
             'ideal_transmitted_bits': 172.22952004916363,
             'packet_efficiency': 1.9230944211497798,
             'packet_redundancy': 25.040719134245172,
             'permeability': 0.8465155784719308,
             'saturation': 15.65068086575483,
             'transmitted_bits': 203.457}}
Found implementation: shuffled_binary_fullrank
  No changes.

{'basic': {'expected_sample_burst_size': 3.01142271939499,
           'expected_sample_data_size': 16.504647865133133,
           'expected_time_to_recover': 13.8861,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.334916273522071,
             'bit_redundancy': 46.17709567122586,
             'ideal_bit_efficency': 0.39602478570730415,
             'ideal_bit_redundancy': 35.463638563486604,
             'ideal_packet_efficiency': 1.9801239285365209,
             'ideal_packet_redundancy': 7.09272771269732,
             'ideal_time_to_recover': 11.74340857845215,
             'ideal_transmitted_bits': 58.71704289226076,
             'packet_efficiency': 1.6745813676103551,
             'packet_redundancy': 9.23541913424517,
             'permeability': 0.8456952332513916,
             'saturation': 4.650680865754831,
             'transmitted_bits': 69.43050000000001}}
Found implementation: simple_matrix
  No changes. Using 

{'basic': {'expected_sample_burst_size': 3.0022058823529414,
           'expected_sample_data_size': 16.559466911764705,
           'expected_time_to_recover': 32.2603,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.15498925924433438,
             'bit_redundancy': 136.3015,
             'ideal_bit_efficency': 0.18308857356913824,
             'ideal_bit_redundancy': 111.54593245579827,
             'ideal_packet_efficiency': 0.9154428678456913,
             'ideal_packet_redundancy': 22.30918649115965,
             'ideal_time_to_recover': 27.30918649115965,
             'ideal_transmitted_bits': 136.54593245579827,
             'packet_efficiency': 0.7749462962216718,
             'packet_redundancy': 27.2603,
             'permeability': 0.8465261169660434,
             'saturation': 5.0,
             'transmitted_bits': 161.3015}}
Found implementation: single_binary_fullrank
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 3.0029218984442867,
           'expected_sample_data_size': 16.442233278054175,
           'expected_time_to_recover': 13.388599999999999,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3473612525398347,
             'bit_redundancy': 43.689595671225845,
             'ideal_bit_efficency': 0.41080146131701784,
             'ideal_bit_redundancy': 33.35156551292115,
             'ideal_packet_efficiency': 2.0540073065850892,
             'ideal_packet_redundancy': 6.670313102584229,
             'ideal_time_to_recover': 11.32099396833906,
             'ideal_transmitted_bits': 56.6049698416953,
             'packet_efficiency': 1.7368062626991736,
             'packet_redundancy': 8.737919134245168,
             'permeability': 0.8455696613790136,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.943}}
Found implementation: single_binary_fullrank2
  No c

{'basic': {'expected_sample_burst_size': 3.00015814027042,
           'expected_sample_data_size': 16.46184865976121,
           'expected_time_to_recover': 13.8109,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.33673988413172423,
             'bit_redundancy': 45.80109567122585,
             'ideal_bit_efficency': 0.3981104461756451,
             'ideal_bit_redundancy': 35.1560258988236,
             'ideal_packet_efficiency': 1.9905522308782257,
             'ideal_packet_redundancy': 7.031205179764719,
             'ideal_time_to_recover': 11.68188604551955,
             'ideal_transmitted_bits': 58.40943022759775,
             'packet_efficiency': 1.6836994206586213,
             'packet_redundancy': 9.16021913424517,
             'permeability': 0.845845386290506,
             'saturation': 4.650680865754831,
             'transmitted_bits': 69.0545}}
